In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import re
import string
from operator import itemgetter
from pandas import *

# ---------------------------------------------------------------------------
# Data Preparation
# ---------------------------------------------------------------------------

Global variables

In [2]:
state_rank_count = 50

Read in opendata_projects file and donation_counts project file

In [3]:
projects = pd.read_csv('../Data/opendata_projects.csv', index_col = False)

In [4]:
donations = pd.read_csv('../Data/donations_counts.csv')

Use "shipping cost" variable to created binary variable indicating whether a project has free shipping

In [5]:
shipping = projects.loc[:,['vendor_shipping_charges']]

# if shipping_charges = 0 , t
# if shipping_charges > 0 , f
shipping[shipping == 0] = -1
shipping[shipping > 0] = 'f'
shipping[shipping == -1] = 't'

shipping.columns = ['free_shipping']

Function to count the number of days between start_date and complete_date

In [6]:
from dateutil.parser import parse
import math
import time

def day_count(date0, date1):
    if isinstance(date0, float) or isinstance(date1, float):
        return 'NaN'
    try:
        delta = parse(date0) - parse(date1)
        delta = delta.days
        if delta < 0:
            delta = 'NaN'       
        elif delta == 0:
            delta = 1      
        else:
            delta = delta
    except Exception, e:
        print date1, type(date1), date0, type(date0)
        raise e
    return delta

Apply dayCount function over dataframe

In [7]:
def date_calc(row):
    return day_count(row['date_completed'], row['date_posted'])
date_diff = projects.apply(date_calc, 1)

Create single vector dataframe of days to completion

In [8]:
days_to_comp = pd.DataFrame({'days_to_completion': pd.Series(date_diff)})

Subset opendata_projects to include only live projects

In [9]:
live_projects = projects[projects.funding_status == 'live']

Add column of current date so we can count open days

In [10]:
live_projects['current_date'] = '2014-11-10'

/home/brianna/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Apply dayCount over live projects

In [11]:
def date_calc2(row):
    return day_count(row['current_date'], row['date_posted'])
date_diff2 = live_projects.apply(date_calc2, 1)

Create single vector dataframe of days open. If project live fewer than 30 days, set days live to NA

In [12]:
days_live = pd.DataFrame({'days_open': pd.Series(date_diff2)})

Add the dataframes containing days_to_completion and days_open to projects dataframe

In [13]:
projects = pd.concat([projects, days_to_comp, days_live, shipping], axis = 1)

Consider all reallocated projects and all projects live for more than 30 days as not funded at all. Delete all remaining live projects (which have been live for less than 30 days)

In [14]:
projects.ix[projects.days_to_completion <= 30, 'funded_by_30'] = 'Yes'
projects.ix[projects.days_to_completion > 30, 'funded_by_30'] = 'No'
projects.ix[projects.funding_status=='reallocated', 'funded_by_30'] = 'No'
projects.ix[projects.days_open > 30, 'funded_by_30'] = 'No'

projects = projects[projects.funded_by_30.notnull()]

Create donor interest parameters by subject and poverty level

By SUBJECT

In [15]:
# Deal with missing values
index = np.where(projects['primary_focus_subject'].isnull())[0]
projects.loc[index, 'primary_focus_subject'] = 'Missing'
total_size = len(projects) 
total_donors = projects['num_donors'].sum()

subjects = projects['primary_focus_subject']
subjects = subjects.unique()
subjects.sort()

In [16]:
num_donors_sub = projects.groupby(['primary_focus_subject']).sum()['num_donors']
subjects_size = projects.groupby('primary_focus_subject').size()
subjects_prop = subjects_size / total_size
scaled_interest_par_sub = (num_donors_sub/total_donors)**2 /subjects_prop 

In [17]:
df = pd.DataFrame({'primary_focus_subject': subjects, 'scaled_interest_par_sub' : scaled_interest_par_sub.values})
projects = pd.merge(projects, df, left_on = 'primary_focus_subject', right_on='primary_focus_subject', how='left')

By POVERTY LEVEL

In [18]:
#No Missing Values
poverty = projects['poverty_level']
poverty = poverty.unique()
poverty.sort()

In [19]:
num_donors_pov = projects.groupby(['poverty_level']).sum()['num_donors']
poverty_size = projects.groupby('poverty_level').size()
poverty_prop = poverty_size / total_size
scaled_interest_par_pov = (num_donors_pov/total_donors)**2 /poverty_prop

In [20]:
df = pd.DataFrame({'poverty_level': poverty, 'scaled_interest_par_pov' : scaled_interest_par_pov.values})
projects_merged = pd.merge(projects, df, left_on = 'poverty_level', right_on='poverty_level', how='left')

In [21]:
#Replace missing values as NaN
projects = projects.replace('Missing', np.nan)

Combine city, state into city_state variable in projects dataframe

In [22]:
projects['city_state'] = projects['school_city'] + ', ' + projects['school_state']

Get the count of the completed projects from each city_state
and create levels from the top 50, and group all remaining into 51st category

In [23]:
comp = projects[projects.funding_status == 'completed']
counts = comp['city_state'].value_counts()
ranks = pd.DataFrame([x+1 for x in range(counts.shape[0])], columns = ['completion_rank'])
names = pd.DataFrame(counts.index, columns = ['city_state'])

In [24]:
ranks = pd.concat([names, ranks], axis = 1)
ranks['city_state_cat'] = ranks['city_state']

In [25]:
ranks['city_state_cat'][ranks.completion_rank > state_rank_count] = 'Other'

/home/brianna/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Join the ranks table and projects table on the city_state columns

In [26]:
projects = projects.merge(ranks, how = 'right', on = 'city_state')

Join the donations table and projects table on the city_state column

In [27]:
projects = projects.merge(donations, how = 'right', on = 'city_state')

Read in outside data

In [28]:
outside_dat =  pd.read_csv('../Data/outside_dat.csv', dtype = {'zip': np.str_, 'med_inc': np.float64, 'pop': np.float64, 'party': np.str_})

Merge outside_data with projects

In [29]:
projects = projects.merge(outside_dat, on = 'school_zip', how='left')

Create test set (~20%) by selecting projects posted from Nov 2013 onwards. Then create training (~60%) and validation (~20%) sets.

Drop last 20986 rows because those rows only have outside data

In [37]:
projects = projects.drop(projects.tail(20986).index)

Save as a data frame for preliminary modeling

# ---------------------------------------------------------------------------
# Generate test set / training set
# ---------------------------------------------------------------------------

Create test set (~20%) by selecting projects posted from Nov 2013 onwards. Then create training (~80%)

In [38]:
def split_train_test(date):
    if isinstance(date, float):
        return 'NaN'
    try:
        date = parse(date)
        day = date.day
        month = date.month
        year = date.year
    except Exception, e:
        print date, type(date)
        raise e
    if year == 2014 or (year == 2013 and (month == 11 or month == 12)):
        return 'Test'
    else:
        return 'Train'

In [39]:
projects['train_test_label'] = [split_train_test(x) for x in projects.date_posted]

train = projects[projects.train_test_label == 'Train']
test = projects[projects.train_test_label == 'Test']

Remove unnecessary columns from projects dataframe

Removed: _projectid, _teacher_acctid, _schoolid, school_ncesid, school_latitude, school_longitude, school_city, school_state, school_zip, school_district, school_county, school_kipp, school_charter_ready_promise, primary_focus_area, secondary_focus_area, vendor_shipping_charges, sales_tax, payment_processing_charges, fulfillment_labor_materials, total_price_including_optional_support, students_reached, total_donations, num_donors, eligible_double_your_impact_match, eligible_almost_home_match, funding_status, date_posted, date_completed, date_thank_you_packet_mailed, date_expiration

In [40]:
drop_cols = ['_projectid', '_teacher_acctid', '_schoolid', 'school_ncesid', 'school_latitude', 'school_longitude',
             'school_city', 'school_state', 'school_zip', 'school_district', 'school_county', 'school_kipp',
             'school_charter_ready_promise', 'primary_focus_area', 'secondary_focus_area', 'vendor_shipping_charges',
             'sales_tax', 'payment_processing_charges', 'fulfillment_labor_materials', 'total_price_including_optional_support',
             'students_reached', 'total_donations', 'num_donors', 'eligible_double_your_impact_match', 
             'eligible_almost_home_match', 'funding_status', 'date_posted', 'date_completed', 'date_thank_you_packet_mailed',
             'date_expiration', 'secondary_focus_subject', 'Unnamed: 0']

In [41]:
projects_modified = projects.drop(drop_cols, axis = 1)

In [42]:
projects_modified.to_csv('../Data/projects_prelim_data.csv')